In [1]:
import pymysql, pandas as pd

## 1st STEP : 2월 User의 data 가져오기 :  TEST01
+ 133.186.135.222:13306 CJ 
+ 133.186.135.21 JJW 

1) 사용자ID + 상점ID + 상점별 결제건수 + 결제합계 ( MEUMS_COMP_REVISIT_HIS)
 - 문제점) 결제 취소정보도 포함되어 있다.
 
2) 라이프존1+라이프존2+라이프존3 ( MEUMS_LATEST_LIFEZONE)

In [2]:
from datetime import datetime


Dt_before = datetime.strptime('2018-02-01', '%Y-%m-%d') 
Dt_after = datetime.strptime('2018-02-28', '%Y-%m-%d') 
dates = list (pd.date_range(Dt_before, Dt_after))
len(dates)

dates[0:1]

[Timestamp('2018-02-01 00:00:00', freq='D')]

In [ ]:
data = pd.DataFrame([])

for i,d in enumerate(dates):
    queryDateFrom = datetime.strftime(d, '%Y-%m-%d' + ' 00:00:00')
    queryDateTo = datetime.strftime(d, '%Y-%m-%d' + ' 23:59:59')
    query = '''SELECT A.EPOPCON_ID, A.COMPANY_ID, count(*) CNT_PAY, sum(A.PAYMENT) SUM_PAY, GROUP_CONCAT(DISTINCT substring_index(A.CARD_NAME, '(', 1)) CARD_NAME 
    FROM MEUMS_COMP_REVISIT_HIS A  WHERE DEAL_DT between "'''+queryDateFrom+'" and "'+queryDateTo+'" GROUP BY A.EPOPCON_ID, A.COMPANY_ID;'
    
    conn_cj = pymysql.connect(host='192.168.0.50', user='eums', password='eums00!q', db='eums-cj-business', charset='utf8')
    curs = conn_cj.cursor(pymysql.cursors.DictCursor)
    curs.execute(query)
    p = pd.DataFrame( curs.fetchall() )
    conn_cj.close()
    
    conn_jjw = pymysql.connect(host='192.168.0.50', user='eums', password='eums00!q', db='eums-business', charset='utf8')
    curs = conn_jjw.cursor(pymysql.cursors.DictCursor)
    curs.execute(query)
    p2 = pd.DataFrame( curs.fetchall() ) 
    p = p.append(p2)
    print(p.shape)
    data=data.append(p)
    
print(data.shape)
data.tail()

In [ ]:
conn_cj = pymysql.connect(host='192.168.0.50', user='eums', password='eums00!q', db='eums-cj-business', charset='utf8')
curs = conn_cj.cursor(pymysql.cursors.DictCursor)
#curs.execute("SELECT * FROM TEST01;")
curs.execute(query)
p = pd.DataFrame( curs.fetchall() ) #[['EPOPCON_ID','COMPANY_ID','CNT_PAY','SUM_PAY']]
conn_cj.close()
p.head()

In [ ]:
conn_jjw = pymysql.connect(host='192.168.0.50', user='eums', password='eums00!q', db='eums-business', charset='utf8')
curs = conn_jjw.cursor(pymysql.cursors.DictCursor)
#curs.execute("SELECT * FROM TEST01;")
curs.execute(query)
p2 = pd.DataFrame( curs.fetchall() ) #[['EPOPCON_ID','COMPANY_ID','CNT_PAY','SUM_PAY']]
p = p.append(p2)
conn_jjw.close()
p2.head()

## 2nd STEP : company data 통합하기 : 통합 서버
: 상점명 + 상점Cate + 상점Cate1 (133.186.143.151:13306 MEUMS_COMPANY) 
 - 문제점) 잘못된 카테고리 정보가 섞여있다.

In [ ]:
conn = pymysql.connect(host='192.168.0.50', user='eums', password='eums00!q', db='eums-shared', charset='utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)
curs.execute("SELECT ID AS COMPANY_ID, ADDR, CO_NAME, CATE, CATE1 FROM MEUMS_COMPANY WHERE STATUS IN (0,1) AND ID IN "+str(tuple(p.COMPANY_ID.unique().reshape(1, -1)[0]))+";")
comp_p = pd.DataFrame( curs.fetchall() )
conn.close()
comp_p.head()

In [ ]:
#p.columns=[['EPOPCON_ID','COMPANY_ID','CNT_PAY','SUM_PAY','LATITUDE1','LONGTIUDE1','CARD_NAME']] ####
p.columns=[['EPOPCON_ID','COMPANY_ID','LATITUDE1','LONGTIUDE1','CARD_NAME','CNT_PAY','SUM_PAY']] ####

Hana_sample = pd.merge(comp_p, p, on='COMPANY_ID',how='inner') # COMPANY 중 STATUS 0,1 아닌 거 제외
Hana_sample = Hana_sample.sort_values(by=['EPOPCON_ID','CATE','CATE1']) ####
Hana_sample['EPOPCON_ID'] = Hana_sample['EPOPCON_ID'].str.replace('cjcis|eppcn','')
print(Hana_sample.shape)
Hana_sample.head()

## 3rd STEP : Export Data to JSON FILE 

In [11]:
#data.to_json('temp.json',orient='records', lines=True) - ,없는 경우
data.to_json('temp.json',orient='records')

import json

with open('temp.json', encoding='utf-8') as data_file:
    data = json.loads(data_file.read()) # type : list

data=pd.DataFrame(data)
data.head()